In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

from scipy.spatial.distance import cdist

import joblib
from sklearn import set_config
from sklearn.metrics  import silhouette_score
set_config(display="diagram")

In [2]:
data = pd.read_csv("../data/final_data.csv", index_col=0)
data.head(5)

,brand,img_url,laptop_name,display_size,processor_type,graphics_card,disk_space,discount_price,ratings_5max,disk_type
0,HP,https://encrypted-tbn0.gstatic.com/images?q=tb...,Notebook 14-df0008nx,14.0,Intel Celeron N4000,Intel HD Graphics 600,64.0,1259.0,0.0,eMMC
1,Lenovo,https://encrypted-tbn0.gstatic.com/images?q=tb...,IdeaPad 330S-14IKB,14.0,Intel Core i5-8250U,Intel UHD Graphics 620,1024.0,1849.0,3.3,HDD
2,Huawei,https://encrypted-tbn0.gstatic.com/images?q=tb...,MateBook D Volta,14.0,Intel Core i5-8250U,NVIDIA GeForce MX150 (2 GB),256.0,2999.0,0.0,SSD
3,Dell,https://encrypted-tbn0.gstatic.com/images?q=tb...,Inspiron 15 3567,15.6,Intel Core i3-7020U,Intel HD Graphics 620,1024.0,1849.0,0.0,HDD
4,Asus,https://encrypted-tbn0.gstatic.com/images?q=tb...,VivoBook 15 X510UR,15.6,Intel Core i7-8550U,NVIDIA GeForce 930MX (2 GB),1024.0,2499.0,0.0,HDD


In [18]:
data_concatenated = data.drop(["img_url", "discount_price", "ratings_5max"], axis=1).T.apply(lambda x: "".join(list(map(str, x.values.tolist())))).to_frame()

In [67]:
data.drop(["img_url", "discount_price", "ratings_5max"], axis=1).T.apply(lambda x: "".join(list(map(str, x.values.tolist()))))

df = data.drop(["img_url", "discount_price", "ratings_5max"], axis=1).T.apply(lambda x: " ".join(list(map(str, x.values.tolist()))))
# data_concatenated["ratings"] = data["ratings_5max"]
# data_concatenated["price"] = data["discount_price"]
df.head()

0    HP Notebook 14-df0008nx 14.0 Intel Celeron N40...
1    Lenovo IdeaPad 330S-14IKB 14.0 Intel Core i5-8...
2    Huawei MateBook D Volta 14.0 Intel Core i5-825...
3    Dell Inspiron 15 3567 15.6 Intel Core i3-7020U...
4    Asus VivoBook 15 X510UR 15.6 Intel Core i7-855...
dtype: object

In [54]:
pd.pivot_table(data_concatenated, values="ratings", index=["data"], columns="data").fillna(0)

data,Acer Aspire 1 A114-31 14.0 Intel Celeron N3350 Intel HD Graphics 500 64.0 eMMC,Acer Aspire 1 A114-31-C6WP 14.0 Intel Celeron N3350 Intel HD Graphics 505 64.0 eMMC,Acer Aspire 3 15.6 Intel Core i3-7020U Intel HD Graphics 620 1024.0 HDD,Acer Aspire 3 A315-33-C6S9 15.6 Intel Celeron N3060 Intel GMA HD 500.0 HDD,Acer Aspire 3 A315-53 15.6 Intel Core i3-7020U Intel HD Graphics 620 1024.0 HDD,Acer Aspire 3 A315-53 15.6 Intel Core i5-7200U NVIDIA GeForce MX130 (2 GB) 1024.0 HDD,Acer Aspire 3 A315-53-341N 15.6 Intel Core i3-7020U Intel Graphics 620 1024.0 HDD,Acer Aspire 3 A315-53-34CE 15.6 Intel Core i3-7020U Intel Graphics 620 1024.0 HDD,Acer Aspire 3 A315-53-52ZL 15.6 Intel Core i5-8250U Intel UHD Graphics 620 16.0 HDD,Acer Aspire 3 A315-53G 15.6 Intel Core i5-8250U NVIDIA GeForce MX130 (2 GB) 1024.0 HDD,...,Lenovo IdeaPad S130-14IGM 14.0 Intel Celeron N4000 Intel UHD Graphics 600 64.0 eMMC,Lenovo IdeaPad S145 15.6 Intel Celeron 4205U Intel GMA HD 500.0 HDD,Lenovo IdeaPad S340 14.0 Intel Core i3-8145U Intel GMA HD 1024.0 HDD,Lenovo IdeaPad S340 14.0 Intel Core i5-8265U Intel GMA HD 1024.0 HDD,Lenovo IdeaPad S340 15.6 Intel Core i5-8265U NVIDIA GeForce MX230 (2 GB) 128.0 HDD,Lenovo IdeaPad S530 13.3 Intel Core i7-8565U NVIDIA GeForce MX150 (2 GB) 512.0 SSD,MSI GT83VR 7RF Titan SLI 18.4 Intel Core i7-7820HK NVIDIA GeForce GTX 1080 (8 GB) SLI 256.0 HDD,Microsoft Surface 13.5 Intel Core i5-7200U Intel HD Graphics 620 256.0 SSD,Microsoft Surface 2 13.5 Intel Core i5-8250U Intel UHD Graphics 620 128.0 SSD,Microsoft Surface 2 13.5 Intel Core i5-8250U Intel UHD Graphics 620 256.0 SSD
data,,,,,,,,,,,,,,,,,,,,,
Acer Aspire 1 A114-31 14.0 Intel Celeron N3350 Intel HD Graphics 500 64.0 eMMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer Aspire 1 A114-31-C6WP 14.0 Intel Celeron N3350 Intel HD Graphics 505 64.0 eMMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer Aspire 3 15.6 Intel Core i3-7020U Intel HD Graphics 620 1024.0 HDD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer Aspire 3 A315-33-C6S9 15.6 Intel Celeron N3060 Intel GMA HD 500.0 HDD,0.0,0.0,0.0,3.7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer Aspire 3 A315-53 15.6 Intel Core i3-7020U Intel HD Graphics 620 1024.0 HDD,0.0,0.0,0.0,0.0,3.7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lenovo IdeaPad S530 13.3 Intel Core i7-8565U NVIDIA GeForce MX150 (2 GB) 512.0 SSD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MSI GT83VR 7RF Titan SLI 18.4 Intel Core i7-7820HK NVIDIA GeForce GTX 1080 (8 GB) SLI 256.0 HDD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Microsoft Surface 13.5 Intel Core i5-7200U Intel HD Graphics 620 256.0 SSD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
import math
import re
from collections import Counter

class Search:
    
    def __init__(self, df):
        self.reg = re.compile(r"\w+")
        self.vectors = df.apply(self.convert_to_vector)     
    
    def convert_to_vector(self, X):
        words = self.reg.findall(X)
        return Counter(words)
    
    def get_cosine(vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
        sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
    
    def search(self, X):
        X = self.convert_to_vector(X)
        res = self.vectors.apply(lambda x: get_cosine(x, X))
        return list(res[res > 0].index)

Cosine: 0.24253562503633297


In [171]:
import math
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class Search:
    
    def __init__(self, df):
        self.reg = re.compile(r"\w+")
        self.vectorizer = CountVectorizer()
        self.df = self.vectorizer.fit_transform(df.values)
    
    def search(self, X):
        X = self.vectorizer.transform([X])
        res = pd.Series([x[0] for x in cosine_similarity(self.df, X)])
        return list(res[res > 0].index)

s = Search(df)
s.search("Lenovo")

[1,
 36,
 46,
 56,
 57,
 60,
 61,
 62,
 64,
 67,
 68,
 73,
 74,
 75,
 119,
 157,
 160,
 161,
 162,
 171,
 187,
 188,
 199]

In [145]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
transformed = vec.fit_transform(df.values)

In [159]:
cosine_similarity(transformed, vec.transform(["Lenovo"]))

array([[0.        ],
       [0.24253563],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.24253563],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.26726124],
       [0.   